In [1]:
import numpy as np
from matplotlib import pyplot as plt
#from matplotlib import rcParams

import plotly
import plotly.graph_objects as go
import os

from sim_handling import Simulation,loadSim,continue_from_file, woa_to_file
from diffusionstuff7 import f0d,f1d,f2d

C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [ ]:
%matplotlib widget

This notebook is to create 3d plots of the number of steps (z axis) over time (y axis) for 1D simulations against a variable parameter (x axis). The parameters to begin with will be those from the 1D QLL mediated continuum model of ice. 
A 3d plot will be created for each parameter. Additionally, the fig11 x value and predicted Nss value will be plotted on a slice of the 3d plot at the final x value, and a difference between the actual and predicted values will printed.
Around 20 values for each parameter will be tested.

In [3]:
#sigma, run and process results
sigma_range = np.linspace(0.19, 0.5, 40)
#Ntots = []
zacc = []

for i,sigma in enumerate(sigma_range):
#set up and run and plot a new simulation
	print(i)
	sigma_sim_2d = Simulation(f2d, (500,500), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sigma_sim_2d.uselayers = False
	sigma_sim_2d.countermax = 20_000 # around 70 layers

	sigma_sim_2d.discretization_halt = False # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	sigma_sim_2d.mem_check = True #manage large simulation
	sigma_sim_2d.memory_threshold = 72E9 #72GB

	#test_1d_asym.float_params['DoverdeltaX2'] 
	sigma_sim_2d.layermax = 1100
	sigma_sim_2d.sigmastepmax = sigma
	sigma_sim_2d.center_reduction = 0.25

	#filename = 'long_2D_sim_500_500_pts.npy'
	filename = 'f2d_500_500_sigma'+str(round(sigma,3))+'.npy'
	sigma_sim_2d.filename = filename

	#run simulation
	run_filename_output = sigma_sim_2d.run()
	need_to_load = False # hope it finished
	while run_filename_output is not None: # if it didn't finish

		#run should just run and save results to Simulation object, outputs filename if it had to stop and save
		print('timesteps completed:', len(sigma_sim_2d._results['t']))
		print('Continuing from file...')
		sigma_sim_2d.countermax -= len(sigma_sim_2d._results['t']) #count completed timesteps
		sigma_sim_2d = continue_from_file(sigma_sim_2d, filename)
		run_filename_output = sigma_sim_2d.run()
		if run_filename_output is None:
			print('Simulation completed')
			woa_to_file(sigma_sim_2d, filename)
			need_to_load = True
			
	if need_to_load:
		print('loading sim results from file')
		Ntot = np.load(filename, mmap_mode='r')[:,1,:,:]
	else:
		print('loading sim results from completed sim object')
		Ntot = sigma_sim_2d.getNtot()

	# maxes = np.array([np.max(n) for n in Ntot[:]])
	# mins = np.array([np.min(n) for n in Ntot[:]])
	#z = np.array(np.max(n)-np.min(n) for n in Ntot) #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)

	# calculate number of steps on the surface
	#zacc.append(np.array([np.max(n.flatten())-np.min(n.flatten()) for n in Ntot[:,:,:]]))
	zacc.append(np.ptp(Ntot, axis=(1,2)))


	print('Ntot shape:', Ntot.shape)
	del Ntot #clear memory of simulation
	del sigma_sim_2d

	#delete saved file to prevent disk space running out
	if os.path.exists(filename):
		os.remove(filename)
	
	with open('zacc.npy', 'wb'): #save number of steps over time list for each sigma to file
		np.save('zacc.npy', zacc)

0


KeyboardInterrupt: 

In [ ]:
print(zacc[0].shape)
print(type(zacc[0]))

In [ ]:
print([np.shape(x) for x in zacc])
print([np.max(n)-np.min(n) for n in zacc])

In [ ]:
my_y = np.asarray(sigma_sim_2d.results()['y'])
#shape of results['y']: (timesteps, 2, 500, 500)

In [ ]:
print(my_y.shape)

In [ ]:
print(type(Ntot))

In [ ]:
import psutil
psutil.swap_memory().free

In [ ]:
np.save('zacc_2d.npy', np.asarray(zacc))

In [ ]:
print(type(zacc[0]))

In [ ]:
zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,sigma_range)

In [ ]:
#matplotlib 3d plot of the results - slow
# fig = plt.figure('number of steps over time vs sigma')
# ax = plt.axes(projection='3d')
# ax.set_xlabel('Sigma')
# ax.set_ylabel('Time')
# ax.set_zlabel('Number of steps (bilyers of ice)')

# print(len(Ntots))

# ax.plot_wireframe(X=xs, Y=ys, Z=zs, cmap='viridis')#, vmin=0, vmax=200)

In [ ]:
#3d plot of results. plotly
fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
fig.update_layout(scene = dict(
                    xaxis_title='Sigma corner',
                    yaxis_title='Time (timesteps)',
                    zaxis_title='Number of steps (bilyers of ice)'),
                    title='number of steps over time vs sigma')
                    

In [ ]:
#Save the plot as an interactive webpage html file
#plotly.offline.plot(fig, filename='Nss_vs_sigma.html')

In [ ]:
#Do the same thing now, but for center reduction

c_r_range = np.linspace(0.01, 4, 40)

#process the results
zacc = []

#run and process results

for center_reduction in c_r_range:
#set up and run and plot a new simulation
	test_1d_asym = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	test_1d_asym.uselayers = False
	test_1d_asym.countermax = 175_000 #enough for steady state under original regular conditions

	test_1d_asym.discretization_halt = False # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	#test_1d_asym.float_params['DoverdeltaX2'] 
	test_1d_asym.layermax = 1100
	test_1d_asym.sigmastepmax = 0.20
	test_1d_asym.center_reduction = center_reduction

	Ntot= test_1d_asym.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,c_r_range)

In [ ]:
#3d plot of results. plotly
fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
fig.update_layout(scene = dict(
                    xaxis_title='Center reduction, %',
                    yaxis_title='Time (timesteps)',
                    zaxis_title='Number of steps (bilyers of ice)'),
                    title='number of steps over time vs center reduction')
fig.show(width=1000, height=1000)

In [ ]:

#save the results as an interactive html file
fig.write_html("c_r_vs_sigma.html")
#plotly.offline.plot(fig, filename='c_r_vs_sigma.html')

In [ ]:
#Do the same thing now, but for Diffusion coefficient
d_range = np.linspace(0.75E-4, 6E-4, 40)

#process the results
zacc = []

#run and process results

for D in d_range:
#set up and run and plot a new simulation
	sim_1d_D = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sim_1d_D.uselayers = False
	sim_1d_D.countermax = 175_000 #enough for steady state under original regular conditions

	sim_1d_D.discretization_halt = False # needs to be same length of simulation, we want to see the differences if it causes discretization halt


	deltaX = sim_1d_D.x[1]-sim_1d_D.x[0]
	sim_1d_D.float_params['DoverdeltaX2'] = D/deltaX**2

	sim_1d_D.layermax = 1100
	sim_1d_D.sigmastepmax = 0.20
	sim_1d_D.center_reduction = 0.25

	Ntot= sim_1d_D.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,d_range)

In [ ]:
#3d plot of results. plotly
fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
fig.update_layout(scene = dict(
                    xaxis_title='D, microns^2/microsecond',
                    yaxis_title='Time (timesteps)',
                    zaxis_title='Number of steps (bilayers of ice)'),
                    title='number of steps over time vs diffusion coefficient')
fig.show(width=1000, height=1000)

In [ ]:
#save the results as an interactive html file
fig.write_html("D_vs_sigma.html")

In [ ]:
#Do the same thing now, but for nu_kin
nu_kin_range = np.linspace(20, 140, 40)

#run and process results
zacc = []

for nu_kin in nu_kin_range:
#set up and run and plot a new simulation
	sim_1d_D = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sim_1d_D.uselayers = False
	sim_1d_D.countermax = 175_000 #enough for steady state under original regular conditions

	sim_1d_D.discretization_halt = False # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	#deprate is accesible, changable parameter, not in microns per second. That value of nu_kin in microns per second is set as 49, so we divide by 49 to get the correct range of deprate values
	sim_1d_D.float_params['deprate'] *= nu_kin/49

	#defaults
	sim_1d_D.layermax = 1100
	sim_1d_D.sigmastepmax = 0.20
	sim_1d_D.center_reduction = 0.25

	Ntot= sim_1d_D.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])
ys,xs = np.meshgrid(ys,nu_kin_range)

In [ ]:
#3d plot of results. plotly
fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
fig.update_layout(scene = dict(
                    xaxis_title='nu_kin, microns/second',
                    yaxis_title='Time (timesteps)',
                    zaxis_title='Number of steps (bilayers of ice)'),
                    title='number of steps over time vs nu_kin')
#

In [ ]:
#save the results as an interactive html file
fig.write_html("nu_kin_vs_sigma.html")

In [ ]:
#Do the same thing now, but for facet size

l_range = np.linspace(150, 850, 40)
l_range = [int(round(l,-1)) for l in l_range] # round to nearest 10

#run and process results
zacc = []
for l in l_range:
#set up and run and plot a new simulation
	sim_1d_D = Simulation(f1d, (l,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sim_1d_D.uselayers = False
	sim_1d_D.countermax = 175_000 #enough for steady state under original regular conditions

	sim_1d_D.discretization_halt = False # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	#deltaX = sim_1d_D.x[1]-sim_1d_D.x[0]
	#sim_1d_D.float_params['DoverdeltaX2'] = D/deltaX**2

	sim_1d_D.layermax = 1100
	sim_1d_D.sigmastepmax = 0.20
	sim_1d_D.center_reduction = 0.25

	Ntot = sim_1d_D.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,l_range)

In [ ]:
#3d plot of results. plotly
fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
fig.update_layout(scene = dict(
                    xaxis_title='L, num points, (10 points per micron)',
                    yaxis_title='Time (timesteps)',
                    zaxis_title='Number of steps (bilayers of ice)'),
                    title='number of steps over time vs facet size')
#

In [ ]:
#save the results as an interactive html file
fig.write_html("L_vs_sigma.html")

In [ ]:
 #set up and run and plot a new simulation
test_1d_asym = Simulation(f1d, (500,), method='RK45', rtol=1e-12)
#test_1d_asym = Simulation(f1d, (500,), method='LSODA')

test_1d_asym.discretization_halt = False

#try a noisy simulation
test_1d_asym.noisy_init = False

#test_1d_asym.float_params['DoverdeltaX2'] *= 2 # raised to lower parameter product value
test_1d_asym.layermax = 1100
#test_2d_asym.sigmastepmax= -0.01 #ablation
test_1d_asym.sigmastepmax = 0.195 #growth - lowered to lower parameter product value
test_1d_asym.sigmastepmax = 0.20 
test_1d_asym.center_reduction = 0.25
#test_2d_asym.center_reduction = 0.003 # lowered to lower parameter product value


In [ ]:
#plot the simulation at the end
test_1d_asym.plot(completion=1, ice=True, tot=True, figurename='ending surface')

In [ ]:
#graph number of steps over time
Ntot = test_1d_asym.getNtot()
#Ntot = cycle_sim.getNtot()
print(Ntot.shape)

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure('Number of steps over time')
ax = plt.subplot(111)

ax.set_ylabel('Number of steps on surface')

if test_1d_asym.sigmastepmax < 0: #ablation case
    ax.set_xlabel('Total number of layers ablated')
    ax.plot( Ntot[::-1,0]+test_1d_asym.layermax,maxes-mins) #flipping to show ablation progress from left to right, adding layermax to show # layers ablated instead of negative # layers grown
else: #growth case   
    ax.set_xlabel('Total number of layers grown')
    ax.plot( Ntot[:,0],maxes-mins)


plt.grid()

In [ ]:
plt.close('Number of steps over time')

In [ ]:
#calculating Nss vs parameter thingy

# N_ss = # number of steps to steady state 
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity


#slope and intercept of linear fit of figure11 x to N_ss
#for sinusoidal growth
M = 0.0027 # bilayers / micron ????
M = 2.7 # what it actually looks like on the plot in the paper
B= 2.9 #bilayers
#for parabolic growth
#M = 0.0025
#B = 1.6

def Nss_fig11(M,B,x):
    return M*x + B

def figure11(L,D,c_r,nu_kin,sigma):
    """
    L: length of domain, microns
    D: diffusion coefficient, microns^2 / microsecond
    c_r: center reduction, fraction
    nu_kin: growth rate, microns / second
    sigma: supersaturation at corner, fraction
    """
    beta = 0.65
    return nu_kin*L**2*sigma*(c_r)**beta / D

#Lx,Ly = test_1d_asym.shape # num points
Lx, = test_1d_asym.shape # num points
Lx_mu = AssignQuantity(25, 'microns')
c_r = test_1d_asym.center_reduction/100
sigma = test_1d_asym.sigmastepmax
beta = 0.65 #value from paper 
nu_kin = AssignQuantity(test_1d_asym._extra_vars.get('nu_kin'), 'micron/second') #microns /second
D = AssignQuantity(test_1d_asym._extra_vars.get('D'),'micron^2 / microsecond')

#print('Lx,Ly',Lx,Ly)
print('Lx',Lx)
print('sigma',sigma)
print('beta',beta)
print('nu_kin',nu_kin)
print('D',D)
print('c_r',c_r)

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print('x with min values from paper: ',x)
print('predicted number of steps at steady state: ', Nss_fig11(M,B,x.magnitude))


#min values from paper
Lx_mu = AssignQuantity(17.5,'microns')
D = AssignQuantity(0.75E-4,'microns^2 / microsecond')
c_r = 0.15/100
nu_kin = AssignQuantity(20,'microns/second')
sigma = 0.20

#print('Lx,Ly',Lx,Ly)
print('Lx',Lx)
print('sigma',sigma)
print('beta',beta)
print('nu_kin',nu_kin)
print('D',D)
print('c_r',c_r)

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print('x with min values from paper: ',x)
print('predicted number of steps at steady state: ', Nss_fig11(M,B,x.magnitude))


#max values from paper
Lx_mu = AssignQuantity(42.5,'microns')
D = AssignQuantity(6E-4,'microns^2 / microsecond')
c_r = 0.7/100
nu_kin = AssignQuantity(140,'microns/second')
sigma = 0.50

#print('Lx,Ly',Lx,Ly)
print('Lx',Lx)
print('sigma',sigma)
print('beta',beta)
print('nu_kin',nu_kin)
print('D',D)
print('c_r',c_r)

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print('x with max values from paper: ',x)
print('predicted number of steps at steady state: ', Nss_fig11(M,B,x.magnitude))

In [ ]:
#calculating parameterization produce (fig 11) for simulation
Lx,Ly = test_1d_asym.shape # num points
sigma = test_1d_asym.sigmastepmax
beta = 0.65 #value from paper 
nu_kin = AssignQuantity(test_1d_asym._extra_vars.get('nu_kin'), 'micron/second') #microns /second
D = AssignQuantity(test_1d_asym._extra_vars.get('D'),'micron^2 / microsecond')
c_r = test_1d_asym.center_reduction

#min values from paper
Lx_mu = AssignQuantity(25,'microns')
Ly_mu = Ly/Lx * Lx_mu

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print(x)

In [ ]:
#plot surface at middle and end of simulation
plot = test_1d_asym.plot(ice=True,completion=0.2)


In [ ]:
plt.close()

In [ ]:
#continuation of previous simulation
#start a second simulation starting with the ending surface of the first 
# as the initial surface

#create 1d sim and graph number of steps over time
cycle_sim =  Simulation(f2d, (test_1d_asym.shape[0],test_1d_asym.shape[1]), method=test_1d_asym.method, rtol=test_1d_asym.rtol)

#keep general params same
cycle_sim.layermax = 1000 #st_2d_asym.layermax * 1.5 #make sure it reaches steady state
cycle_sim.float_params['DoverdeltaX2'] = test_1d_asym.float_params['DoverdeltaX2'] 
cycle_sim.sigma0 = test_1d_asym.sigma0
cycle_sim.center_reduction = test_1d_asym.center_reduction
cycle_sim.deltaT = test_1d_asym.deltaT #increase time step to decrease number of steps saved?
cycle_sim.noisy_init = test_1d_asym.noisy_init

#set up initial surface
cycle_sim.nonstd_init = True
ntots = test_1d_asym.getNtot()
cycle_sim.startingNtot = ntots[len(ntots)//2] #start with the middle of the last simulation, before discretzation error

#set to ablation run
cycle_sim.sigmastepmax = 0.20 


In [ ]:
#animate ablation of growth ending surface
anim = cycle_sim.animate(ret=True, focus_on_growth=True, proportionalSpeed=False)

In [ ]:
#graph number of steps over time for second simulation
Ntot = cycle_sim.getNtot()
#Ntot = cycle_sim.getNtot()

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure()
ax = plt.subplot(111)

ax.set_ylabel('Number of steps on surface')

if cycle_sim.sigmastepmax < 0: #ablation case
    ax.set_xlabel('Total number of layers ablated')
    ax.plot( Ntot[::-1,0,0]+cycle_sim.layermax,maxes-mins) #flipping to show ablation progress from left to right, adding layermax to show # layers ablated instead of negative # layers grown
else: #growth case   
    ax.set_xlabel('Total number of layers grown')
    ax.plot( Ntot[:,0,0],maxes-mins)

plt.grid()